In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM, PeftModel
from cut_cross_entropy.transformers import cce_patch
from cut_cross_entropy import linear_cross_entropy
import torch
import transformers
import numpy as np
import random
from torchviz import make_dot

In [3]:
tokenizer = AutoTokenizer.from_pretrained('HuggingFaceTB/SmolLM2-135M-Instruct')

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    'HuggingFaceTB/SmolLM2-135M-Instruct',
    torch_dtype = torch.bfloat16
).cuda()

In [5]:
rank = 128
peft_config = LoraConfig(
        lora_alpha=rank * 2,
        lora_dropout=0.0,
        r=rank,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["embed_tokens", "lm_head"],
    )

In [6]:
model = get_peft_model(model, peft_config)

/home/husein/.local/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:543: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [7]:
input_ids = torch.tensor([1,2,3])[None].cuda()

In [8]:
o = model(input_ids = input_ids, output_hidden_states = True)

In [9]:
classifier = model.lm_head.weight

In [10]:
classifier.shape

torch.Size([49152, 576])

In [11]:
c_a = model.lm_head.lora_A.default.weight
c_b = model.lm_head.lora_B.default.weight
alpha = model.lm_head.scaling['default']

In [12]:
manual_shift_loss = linear_cross_entropy(o.hidden_states[-1], classifier,
                                         c_a = c_a,
                                         c_b = c_b,
                                         alpha = alpha,
                                         targets = input_ids, shift = True)

In [13]:
manual_shift_loss

tensor(19.7411, device='cuda:0', grad_fn=<LinearCrossEntropyFunctionBackward>)

In [14]:
model(input_ids = input_ids, labels = input_ids).loss

tensor(19.7810, device='cuda:0', grad_fn=<NllLossBackward0>)